In [1]:
import open3d as o3d
import copy
import math
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import time

In [2]:
import math
import numpy as np 

def readData(filename):
    data = np.loadtxt(filename)
    #data[i].reshape(3,4)
    return data 

def readPointCloud(filename):
    pcl = np.fromfile(filename, dtype=np.float32,count=-1)
    pcl = pcl.reshape([-1,4])
    return pcl 

def EulerAngles_To_RotationMatrix(theta) :    
    R_x = np.array([[1,0,0],
                    [0,math.cos(theta[2]),-math.sin(theta[2]) ],
                    [0,math.sin(theta[2]),math.cos(theta[2])  ]
                    ])
    R_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])  ],
                    [0,1,0],
                    [-math.sin(theta[1]),0,math.cos(theta[1]) ]
                    ])
    R_z = np.array([[math.cos(theta[0]),-math.sin(theta[0]),0 ],
                    [math.sin(theta[0]),math.cos(theta[0]),0],
                    [0,0,1]
                    ])
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R
def unique_rows(a):
    a = np.ascontiguousarray(a)
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))

In [3]:
import matplotlib

my_mat=readData('../dataset/01.txt')
final = o3d.geometry.PointCloud()
for i in range (77):
    st=[0,0,0,0,0,0]
    
    for j in range (6):
        st[5-j]=math.floor((i/(10**j))%10)
        
    st_me=''.join([str(elem) for elem in st]) 
    myfile="../dataset/01/"+st_me+".bin"
    yy=readPointCloud(myfile)
    me=EulerAngles_To_RotationMatrix([math.radians(90),math.radians(-90),0])
    
    me2=np.zeros((3,4))
    for p in range(3):
        for q in range(3):
            me2[p][q]=me[p][q]
    yt=np.transpose(yy)
    yy=np.matmul(me2,yt)
    
    me2=yy.shape[1]
    yy.resize((4,me2))
    temp=my_mat[i]
    temp2=np.zeros((3,4))
    yy[3]=np.ones((1,me2))
    for r in range(3):
        for s in range(4):
            temp2[r][s]=temp[(r*4)+s]
            
    yp=np.matmul(temp2,yy)
    yt1=np.transpose(yp)
    y_rn = np.rint(yt1)
    y_rnd = y_rn.astype(int)
    
    mini  =  np.min(y_rnd,axis=0)
    maxi  =  np.max(y_rnd,axis=0)
    
    grid = np.zeros((abs(mini[0])+maxi[0]+100,abs(mini[2])+maxi[2]+100 ))
 
    final = unique_rows(y_rnd).astype(int)
    
    new = np.zeros((final.shape[0],2))
    for re in range(final.shape[0]):
        new[re][0]=final[re][0]
        new[re][1]=final[re][2]
    
    arr  = [tuple(row) for row in new]
    
    values, occur = np.unique(arr, return_counts=True,axis=0)
    threshold = 2
    c=0
    for v in range(values.shape[0]):
        if(occur[v]>=threshold):
            grid[int(values[v][1]+abs(mini[2]))][int(values[v][0]+abs(mini[0]))]=1
    string = "grid_"+st_me+".png"
    matplotlib.image.imsave(string, grid,cmap='gray')

    if(i==0):
        chec=yt1
    else :
        if((i%5)==0 and i<17):
            
            ch_n    = np.rint(chec)
            check = ch_n.astype(int)
            mino  =  np.min(check,axis=0)
            maxo  =  np.max(check,axis=0)
    
            grid1 = np.zeros(((abs(mino[0])+maxo[0]+50),(abs(mino[2])+maxo[2]+50) ))
 
            final1 = unique_rows(check).astype(int)
    
            new1 = np.zeros((final1.shape[0],2))
            for rey in range(final1.shape[0]):
                new1[rey][0]=final1[rey][0]
                new1[rey][1]=final1[rey][2]
    
            arr1  = [tuple(row) for row in new]
    
            vales, ocur = np.unique(arr1, return_counts=True,axis=0)
            threshol = 3
            for v1 in range(vales.shape[0]):
                if(ocur[v1]>=threshol):
                    grid1[int(vales[v1][1]+abs(mino[2]))][int(vales[v1][0]+abs(mino[0]))]=1
            string = "grid_"+str(i)+"_scans"+".png"
            matplotlib.image.imsave(string, grid1,cmap='gray')
        
        chec = np.concatenate((chec,yt1),axis=0)
            